# Sistema de Identificação e Recomendação.
---

# Carregando a base de dados artificial:


In [1]:
!pip install openpyxl

In [2]:
import openpyxl # This line imports the necessary library for reading xlsx files

In [3]:
from google.colab import drive
import pandas as pd
import os

# Montar o Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
caminho_base = '/content/drive/MyDrive/Ciência da Computação/Hackaton/Bert/Base.xlsx'
caminho_culturas_pragas = '/content/drive/MyDrive/Ciência da Computação/Hackaton/Bert/Culturas_e_pragas.xlsx'

df_base = pd.read_excel(caminho_base, engine='openpyxl')
df_culturas_pragas = pd.read_excel(caminho_culturas_pragas, engine='openpyxl')

In [5]:
df_base.head()

print(df_base.shape)

(50, 11)


In [6]:
df_culturas_pragas.head()

print(df_culturas_pragas.shape)

(12, 6)


# A partir daqui trabalhando com os Embedings e com o Gradio

In [7]:
!pip install transformers
!pip install torch
!pip install transformers gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [9]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import gradio as gr

# Carregar os dados das planilhas
df_base = pd.read_excel(caminho_base, sheet_name=0, engine='openpyxl')
df_culturas_pragas = pd.read_excel(caminho_culturas_pragas, sheet_name=0, engine='openpyxl')

# Inicializar o tokenizer e o modelo BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Obter lista de perguntas
perguntas = df_base.columns[:-1].tolist()  # Todas as perguntas, exceto a coluna "DIAGNÓSTICO"

# Função para calcular embeddings de uma lista de textos
def calcular_embeddings(textos):
    entradas = tokenizer(textos, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**entradas)
    return outputs.last_hidden_state[:, 0, :]

# Pré-calcular os embeddings dos diagnósticos
diagnosticos = df_base['DIAGNÓSTICO'].tolist()
embeddings_diagnosticos = calcular_embeddings(diagnosticos)

# Função de diagnóstico
def diagnosticar(respostas_usuario):
    # Calcular embedding médio das respostas do usuário para as 10 perguntas
    embeddings_respostas = calcular_embeddings(respostas_usuario)
    embedding_medio_respostas = embeddings_respostas.mean(dim=0, keepdim=True)

    # Comparar o embedding médio das respostas com cada embedding de diagnóstico
    similaridades = torch.nn.functional.cosine_similarity(embedding_medio_respostas, embeddings_diagnosticos)
    indice_diagnostico = similaridades.argmax().item()
    diagnostico = df_base.loc[indice_diagnostico, 'DIAGNÓSTICO']

    # Buscar tratamentos na segunda planilha
    praga = df_culturas_pragas[df_culturas_pragas['Categoria da Praga'] == diagnostico]
    if not praga.empty:
        tratamento_nivel_1 = praga.iloc[0]['Tratamento Nível 1 (Orgânico)']
        tratamento_nivel_2 = praga.iloc[0]['Tratamento Nível 2 (Genérico)']
        tratamento_nivel_3 = praga.iloc[0]['Tratamento Nível 3 (Agrotóxico Controlado)']
    else:
        tratamento_nivel_1 = tratamento_nivel_2 = tratamento_nivel_3 = "Tratamento não encontrado"

    return diagnostico, tratamento_nivel_1, tratamento_nivel_2, tratamento_nivel_3

# Interface Gradio
def interface_gradio(*respostas_usuario):
    respostas_usuario = list(respostas_usuario)
    diagnostico, tratamento1, tratamento2, tratamento3 = diagnosticar(respostas_usuario)
    return [diagnostico, tratamento1, tratamento2, tratamento3]

inputs = [gr.Textbox(label=pergunta) for pergunta in perguntas]
outputs = [
    gr.Textbox(label="Diagnóstico"),
    gr.Textbox(label="Tratamento Nível 1 (Orgânico)"),
    gr.Textbox(label="Tratamento Nível 2 (Genérico)"),
    gr.Textbox(label="Tratamento Nível 3 (Agrotóxico Controlado)")
]

gr.Interface(
    fn=interface_gradio,
    inputs=inputs,
    outputs=outputs,
    title="Diagnóstico e Tratamento de Pragas",
    description="Responda às perguntas para obter um diagnóstico e as opções de tratamento."
).launch()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3bcc7fa6a32bb4488b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
